In [1]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os 

In [2]:
from tensorflow.python.keras.callbacks import TensorBoard

In [15]:
from time import time

In [3]:
train_path = 'G:\\PYTHON\\Final year project\\Dataset\\train'
val_path = 'G:\\PYTHON\\Final year project\\Dataset\\val'
test_path = 'G:\\PYTHON\\Final year project\\Dataset\\test'
#test_path= test_data

In [4]:
# re-size all the images to a size VGG-16 expects.
IMAGE_SIZE = [224, 224]

# Set the batch size
BATCH_SIZE = 16  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001
NUM_CLASSES = 2 # We are aware of it.

In [5]:
CLASSES = os.listdir(train_path)
NUM_CLASSES = len(CLASSES)
print("Class --> {} \n and the length is : {}".format(CLASSES, NUM_CLASSES))

Class --> ['covid', 'normal'] 
 and the length is : 2


In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [7]:
# Import the images from the train dataset.
# Make sure to provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 6030 images belonging to 2 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
# Import the images from the test dataset.

test_set = test_datagen.flow_from_directory(
    directory = test_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 860 images belonging to 2 classes.


In [10]:
vgg = VGG19(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 8s 0us/step


In [11]:
for layer in vgg.layers:
    layer.trainable = False

In [12]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

prediction = Dense(NUM_CLASSES, activation='softmax')(x)

In [13]:
model = Model(inputs=vgg.input, outputs=prediction)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
Tensorboard = TensorBoard(log_dir="G:\\PYTHON\\Final year project\\Logs\\{}".format(time()))

In [17]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
%%timeit -n1 -r1 
with tf.device('/GPU:0'):
    history = model.fit(
        training_set,
        validation_data=test_set,
        epochs=20,
        steps_per_epoch=len(training_set),
        validation_steps=len(test_set),
        callbacks=[Tensorboard]
    )

Epoch 1/20
377/377 [==============================] - 121s 293ms/step - loss: 0.4455 - accuracy: 0.8378 - val_loss: 0.5023 - val_accuracy: 0.7651
Epoch 2/20
377/377 [==============================] - 93s 246ms/step - loss: 0.2688 - accuracy: 0.8924 - val_loss: 0.5607 - val_accuracy: 0.7977
Epoch 3/20
377/377 [==============================] - 99s 262ms/step - loss: 0.2668 - accuracy: 0.8990 - val_loss: 0.6942 - val_accuracy: 0.7663
Epoch 4/20
377/377 [==============================] - 184s 487ms/step - loss: 0.2412 - accuracy: 0.9144 - val_loss: 0.3343 - val_accuracy: 0.8477
Epoch 5/20
377/377 [==============================] - 159s 423ms/step - loss: 0.2044 - accuracy: 0.9235 - val_loss: 0.3458 - val_accuracy: 0.8547
Epoch 6/20
377/377 [==============================] - 116s 308ms/step - loss: 0.2190 - accuracy: 0.9159 - val_loss: 0.5568 - val_accuracy: 0.7849
Epoch 7/20
377/377 [==============================] - 123s 326ms/step - loss: 0.1947 - accuracy: 0.9250 - val_loss: 0.3357 - v

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  MemoryError: Unable to allocate 9.19 MiB for an array with shape (16, 224, 224, 3) and data type float32
Traceback (most recent call last):

  File "C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\tensorflow\python\ops\script_ops.py", line 249, in __call__
    ret = func(*args)

  File "C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 892, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\keras\engine\data_adapter.py", line 822, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\Charan Chandra\anaconda3\envs\gputest\lib\site-packages\keras\en

In [19]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

In [20]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 1726 images belonging to 2 classes.


In [21]:
validation_steps = 200

loss0,accuracy0 = model.evaluate(validation_set, steps = validation_steps)

print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

200/200 [==============================] - 21s 106ms/step - loss: 0.2966 - accuracy: 0.8969
loss: 0.30
accuracy: 0.90


In [22]:
# Generate Validation set.
validation_set2 = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = 1,
    shuffle=False, 
    seed=42, 
    class_mode="binary"
)

# validation_set2.reset()

Found 1726 images belonging to 2 classes.


In [23]:
# just capture the loss and accuray into val variable... unlike in pervious code to capture into loss0 and accuracy0. Just to showcase alternate way.

test = model.evaluate(test_set)

print("loss: {:.2f}".format(test[0]))
print("accuracy: {:.2f}".format(test[1]))

54/54 [==============================] - 8s 145ms/step - loss: 0.3784 - accuracy: 0.8674
loss: 0.38
accuracy: 0.87


In [27]:
model.save("G:\\PYTHON\\Final year project\\weights\\vgg_19_weights.h5")